In [84]:
from collections import Counter
from typing import List, Tuple
import math
import nltk
from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger
from nltk.corpus import stopwords as nltk_stopwords

try:
    nltk_stopwords.words("russian")
except LookupError:
    nltk.download("stopwords")

In [85]:
text = "Я бы смог завоевать весь мир одной рукой, если бы ты держала вторую. Очнись и взгляни на реальность,не всегда все идет по плану. Чем дольше ты живешь тем больше начинаешь понимать это. В этой бесполезной реальности существует лишь боль,ненависть и страдание. Слушай внимательно.....В этом мире везде где есть свет,всегда есть и тень. Пока в этом мире есть победители будут и проигравшие а эгоистичное желание мира всегда ведет к войне. Из желания защитить любимых людей-рождается ненависть."

segmenter = Segmenter()
morph_vocab = MorphVocab()
embedding = NewsEmbedding()
morph_tagger = NewsMorphTagger(embedding)

russian_stopwords = set(nltk_stopwords.words("russian"))

# делим текст на параграфы
def split_text_to_sentences(text: str) -> List[str]:
    doc = Doc(text)
    doc.segment(segmenter)
    return [sent.text.strip() for sent in doc.sents if sent.text.strip()]


def group_sentences(sentences: List[str], group_size: int = 200) -> List[str]:
    return [" ".join(sentences[i:i + group_size]) for i in range(0, len(sentences), group_size)]


def preprocess_text(paragraphs: List[str]) -> List[List[str]]:
    result = []
    for paragraph in paragraphs:
        doc = Doc(paragraph)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)

        tokens = []
        for token in doc.tokens:
            if token.pos in ("PUNCT", "NUM"):
                continue
            token.lemmatize(morph_vocab)
            lemma = token.lemma.lower()
            if lemma not in russian_stopwords:
                tokens.append(lemma)
        result.append(tokens)
    return result

print("preprocessed_text: ", preprocessed_paragraphs[0][:20])

preprocessed_text:  ['смочь', 'завоевать', 'весь', 'мир', 'рука', 'держать', 'второй', 'очнуться', 'взглянуть', 'реальность', 'весь', 'идти', 'план', 'долгий', 'жить', 'большой', 'начинать', 'понимать', 'это', 'бесполезный']


In [86]:
def get_vocabulary(docs: List[List[str]]) -> List[str]:
    return sorted(set(token for doc in docs for token in doc))

In [87]:
# bag_of_words


def bag_of_words(text: list[str]) -> list[int]:
    vocabulary = get_vocabulary(text)

    bow_matrix = []
    for tokens in text:
        row = [0] * len(vocabulary)
        for token in tokens:
            if token in vocabulary:
                j = vocabulary.index(token)
                row[j] += 1
        bow_matrix.append(row)

    return bow_matrix


matrix = bag_of_words(preprocessed_paragraphs[:10])
print("Bag of Words")
print("matrix: ", matrix[0][:100])

Bag of Words
matrix:  [1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 4, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
# tf_idf

def compute_df(
    vocabulary: list[str],
    docs: list[list],
) -> list[int]:
    df: list[int] = [0] * len(vocabulary)
    for i, word in enumerate(vocabulary):
        for tokens in docs:
            if word in tokens:
                df[i] += 1
    return df


def compute_idf(
    df: list[int],
    N: int,
) -> list[float]:
    idf: list[float] = [math.log(N / (1 + df_val)) for df_val in df]
    return idf


def compute_tf(
    doc: list[str],
    vocabulary: list[str],
) -> list[float]:
    total_words: int = len(doc)
    word_counts: dict[str, int] = {}
    for token in doc:
        word_counts[token] = word_counts.get(token, 0) + 1

    tf: list[float] = []
    for word in vocabulary:
        count: int = word_counts.get(word, 0)
        tf_val: float = count / total_words if total_words > 0 else 0
        tf.append(tf_val)
    return tf


def compute_tf_idf_for_doc(
    doc: list[str],
    vocabulary: list[str],
    idf: list[float],
) -> list[float]:
    tf: list[float] = compute_tf(doc, vocabulary)
    tf_idf: list[float] = [tf_val * idf_val for tf_val, idf_val in zip(tf, idf)]
    return tf_idf


def tf_idf(docs: list[str]) -> tuple:
    vocabulary = get_vocabulary(docs)
    N = len(docs)
    df = compute_df(vocabulary, docs)
    idf = compute_idf(df, N)

    tfidf_matrix: list[list] = []
    for doc in docs:
        tfidf_vector: list[float] = compute_tf_idf_for_doc(doc, vocabulary, idf)
        tfidf_matrix.append(tfidf_vector)

    return vocabulary, tfidf_matrix


vocab_tfidf, tfidf_matrix = tf_idf(preprocessed_paragraphs)
print("TF-IDF Словарь:")
print(vocab_tfidf[:20])
print("Матрица TF-IDF:")
print(tfidf_matrix[0][:20])

TF-IDF Словарь:
['бесполезный', 'боль', 'большой', 'везде', 'вести', 'весь', 'взглянуть', 'внимательно', 'война', 'второй', 'держать', 'долгий', 'желание', 'жить', 'завоевать', 'защитить', 'идти', 'лишь', 'любимый', 'людей-рождаться']
Матрица TF-IDF:
[-0.015068416968694463, -0.015068416968694463, -0.015068416968694463, -0.015068416968694463, -0.015068416968694463, -0.030136833937388925, -0.015068416968694463, -0.015068416968694463, -0.015068416968694463, -0.015068416968694463, -0.015068416968694463, -0.015068416968694463, -0.030136833937388925, -0.015068416968694463, -0.015068416968694463, -0.015068416968694463, -0.015068416968694463, -0.015068416968694463, -0.015068416968694463, -0.015068416968694463]
